# 1. 分类数据
- 表征重复值，以进行高效的存储和计算
- 分类或字典编码表示法：dim.take(val)
    - 按照val对dim进行索引
    - dim: 维表，包含所有不同元素的Series
    - val: 整数键，对dim的索引值的Series
- pandas的分类类型category
    - data.astype('category')
    - data.astype('category').values 是一个pd.Categorical实例，该对象有2个属性categories和codes（返回结果分别类似dim和val）
    - 可直接创建pd.Categorical([...])
    - 可根据分类编码构造pd.Categorical.from_codes(codes, categories)
        - pd.Categorical.from_codes(codes, categories).as_ordered()
        - 传入ordered=True，指定一个有意义的顺序
- 用分类进行计算
    - 与非编码（如字符串数组）类似
    - data.groupby(pd.qcut(data, n, labels=)).agg('func').reset_index()
- 分类方法
    - 查看cat属性：c.cat.codes、c.cat.categories
    - 修改分类集（可改名称和数目）：c.cat.set_categories(new_categories)
    - 修改分类名称，但不能改变分类数目：c.cat.rename_categories(新名称)
    - 将原分类元素重新排序：c.cat.reorder_categories(新顺序)
    - 使分类有序：c.cat.as_ordered()
    - 使分类无序：c.cat.as_unordered()
    - 截取部分分类元素：c[c.isin([...])]
    - 删除结果中未出现的分类元素：c.cat.remove_unused_categories()
    - 删除指定的分类元素：c.cat.remove_categories(指定元素)
    - 在已存在的分类后面添加新的分类：add_categories(新元素)
- 为建模创建虚拟变量
    - 使用统计或机器学习时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。包括创建一个不同类别的列的DataFrame，这些列包含给定分类的1，其余为0
    - pd.get_dummies(s)

# 2. GroupBy高级应用
## 分组转换和解封grouped.transform(func)
- transform与apply相似，但对使用的函数有一定限制
- 可以产生一个和输入组形状相同的对象，而不是对相同组进行合并（apply也会合并）；
- 对于内置的聚合函数，可以向transform传递函数名，而apply不可以；
- 可以产生向分组形状广播标量值；不能修改输入；
- 解封分组：内置聚合函数，通常比apply函数和transform快，因此尽量直接使用内置函数进行运算。

## 分组的时间重采样
- 对于时间序列数据，resample方法从语义上是一个基于内在时间的分组操作；
- 对于包含多个时间序列的DataFrame，若想用一个额外的分组键的列进行标记，则需对每个key值进行相同的重采样，可引入pd.Grouper对象：resampled = df.set_index(...).groupby(['key', pd.Grouper(freq='5min')]).sum()
- TimeGrouper的限制是时间必须是Series或DataFrame的索引。

## 链式编程技术
- 没看懂

In [43]:
# GroupBy高级应用-链式编程技术

import numpy as np
import pandas as pd

df = pd.DataFrame({'key': list('abcd'),
                   'col1': np.arange(4),
                   'col2': [-1, 2.2, -1.3, 3]})
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col2'].mean()
result = df2.groupby('key').col1_demeaned.std()

df3 = df.copy()
df3['k

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,key,col1,col2
0,a,0,-1.0
1,b,1,2.2
2,c,2,-1.3
3,d,3,3.0


In [75]:
# 分类
import numpy as np
import pandas as pd

values = pd.Series(['apple', 'orange', 'apple'] * 2)
pd.unique(values)
pd.value_counts(values)


#维表
val = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
dim.take(val)


#分类类型category
fruits = ['apple', 'orange', 'apple'] * 2
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(len(fruits)),
                   'count': np.random.randint(3, 15, size=len(fruits)),
                   'weight': np.random.uniform(0, 4, size=len(fruits))},
                   columns=['basket_id', 'fruit', 'count', 'weight'])

df['fruit'].values

df['fruit'].astype('category')
c = df['fruit'].astype('category').values 
type(c)  #category的值不再是np数组，而是一个pd.Categorical实例
c.categories
c.codes


#直接创建
my_cat = pd.Categorical(['baz', 'bar', 'baz', 'foo', 'bar'])
my_cat
#根据分类编码构造
categories = ['baz', 'bar', 'foo']
codes = [2, 1, 0, 2, 1]
my_cat2 = pd.Categorical.from_codes(codes, categories)
my_cat2
my_cat2.as_ordered() #排序
pd.Categorical.from_codes(codes, categories, ordered=True) #指定顺序


#用分类进行计算
draws = np.random.randn(1000)
draws[:5]
bins = pd.qcut(draws, 4)
bins2 = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins3 = pd.Series(bins2, name='quartile')
results = pd.Series(draws).groupby(bins3).agg(['count', 'min', 'max']).reset_index()
results
results['quartile']


#用分类提高性能
N = 10000000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
categories = labels.astype('category')
#labels.memory_usage()
#categories.memory_usage()

#%time _ = labels.astype('category')


#分类方法
s = pd.Series(['a', 'b', 'c', 'd'] * 2).astype('category')
s.values.codes
s.cat.codes
s.cat.categories

categories2 = ['a', 'b', 'e', 'd', 'c']
s2 = s.cat.set_categories(categories2)  #修改分类集
s2

s.value_counts()
s2.value_counts()

s3 = s[s.isin(['a', 'b'])]  #截取部分分类元素
s3
s3.cat.remove_unused_categories()  #删除结果中未出现的分类元素

s3.cat.remove_categories('a')  #删除选定的分类元素
s3.cat.add_categories(['f'])  #添加新分类
s2.cat.as_ordered()  #使分类有序
s2.cat.as_unordered()  #使分类无序
s3.cat.rename_categories(['a1', 'b1', 'c1', 'd1'])  #替换分类名字，不能改变数目
s3.cat.reorder_categories(['b', 'a', 'd', 'c'])  #将原分类元素重新排序
s3.cat.set_categories(['a2', 'b2', 'c2'])  #替换分类名字，可以改变数目


#为建模创建虚拟变量
ss = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
pd.get_dummies(ss)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


In [ ]:
# GroupBy高级应用-transform

import pandas as pd
import numpy as np

df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12)})
g = df.groupby('key').value
g.mean()
g.transform('mean')
g.transform(lambda x: x.mean())
g.transform(lambda x: x * 2)
g.transform(lambda x: x.rank(ascending=False))

#transform与apply类似
def normalize(x):
    return (x - x.mean()) / x.std()

g.transform(normalize)
g.apply(normalize)

g.apply(lambda x: x.mean()) #与transform不同

#解封分组操作
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

In [52]:
# GroupBy高级应用-分组的时间重采样

import numpy as np
import pandas as pd

N = 15
times = pd.date_range('2020-05-22 00:00', freq='1min', periods=15)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df.set_index('time').resample('5min').count()

df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3)})
time_key = pd.Grouper(freq='5min')
time_key
resampled = df2.set_index('time').groupby(['key', time_key]).sum()
resampled.reset_index()

,key,time,value
0,a,2020-05-22 00:00:00,30
1,a,2020-05-22 00:05:00,105
2,a,2020-05-22 00:10:00,180
3,b,2020-05-22 00:00:00,35
4,b,2020-05-22 00:05:00,110
5,b,2020-05-22 00:10:00,185
6,c,2020-05-22 00:00:00,40
7,c,2020-05-22 00:05:00,115
8,c,2020-05-22 00:10:00,190


In [44]:
pd.Grouper('5min')

Grouper(key='5min', axis=0, sort=False)